In [ ]:
#install LP solver library
!pip install pulp
from pulp import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 51.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

KeyboardInterrupt: ignored

In [ ]:
x = LpVariable("x", 0, 3)
y = LpVariable("y", 0, 1)
prob = LpProblem("myProblem", LpMaximize)
prob += -4*x + y
prob += x + y <= 2

In [ ]:
status = prob.solve()
print(LpStatus[status])
print(value(y))

obj = value(prob.objective)
print(obj)

Optimal
1.0
1.0


In [ ]:
# TSP Solver

# install tsp package
!pip install python-tsp

import numpy as np
import pandas as pd
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.distances import tsplib_distance_matrix
from python_tsp.heuristics import solve_tsp_local_search, solve_tsp_simulated_annealing

# define a function to remove some points in dataset
def mis(i):
    tsplib_file = "/content/a10.tsp"
    # construct distance matrix
    distance_matrix = tsplib_distance_matrix(tsplib_file)

    # exclude ith customer
    distance_matrix = np.delete(distance_matrix, i, axis=1)
    distance_matrix = np.delete(distance_matrix, i, axis=0)
    # run the SA with a 2-opt and follow it by a LS with PS3
    permutation, distance = solve_tsp_simulated_annealing(distance_matrix)
    permutation, distance = solve_tsp_local_search(distance_matrix,
                                                   x0 = permutation,
                                                   perturbation_scheme = "ps3")

    return [permutation, distance]

print("Total distance from depot 1 is", mis(1)[1])
print("Total distance from depot 2 is", mis(0)[1])

# compute DC_i of  depot 1 and depot 2 from group P
x = [0]*8
y = [0]*8
for i in range(8):
    k = [1, i+2]
    s = [0, i+1]
    x[i] = mis(k)[1]
    y[i] = mis(s)[1]

print(x)
print(y)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Total distance from depot 1 is 3166
Total distance from depot 2 is 3227
[2457, 2983, 3050, 3101, 3076, 3121, 3128, 2851]
[3137, 3023, 3044, 3111, 3162, 2873, 3182, 3133]


In [ ]:
import csv


file = open("coordinates10.csv", "r")
data = list(csv.reader(file))
file.close()
for i in range(len(data)):
    for j in range(len(data[i])):
        data[i][j] = int(data[i][j])

for i in range(2, len(data)):
    data[i] = [data[i][0], data[i][1], data[i][2], data[i][3:6], data[i][6:]]
print(data)
#[[x, y, max_inventory, [1,2,3(inventory)], [1,2,3 (demand)]], [customer 2], [customer 3]...]

[[124, 803], [975, 265], [943, 81, 200, [130, 65, 0], [195, 130, 65]], [546, 287, 70, [70, 35, 35], [35, 35, 35]], [353, 20, 100, [58, 58, 0], [58, 58, 58]], [209, 108, 50, [48, 24, 24], [24, 24, 24]], [228, 684, 60, [11, 0, 22], [44, 22, 33]], [234, 426, 300, [140, 210, 0], [280, 280, 140]], [663, 439, 150, [30, 0, 0], [120, 30, 30]], [625, 893, 800, [600, 200, 400], [400, 400, 400]]]


In [ ]:
P = data[0:2]
I = data[2:]
T = [0, 1, 2]

R = [0 for i in range(8)]
M = [0 for i in range(8)]
CL = [0 for i in range(8)]

alpha = 0.5

for i in range(len(I)):
    max_inventory = I[i][2]
    initial_inventory = I[i][3][0]
    sum = 0
    for t in T:
        sum += I[i][4][t] - initial_inventory
    if sum > 0:
        R[i] = sum / max_inventory
    else:
        R[i] = 0

TSP_all = [3166, 3227]
TSP = [[2457, 2983, 3050, 3101, 3076, 3121, 3128, 2851], [3137, 3023, 3044, 3111, 3162, 2873, 3182, 3133]]
cost = 0
for i in range(len(I)):
    for p in range(len(P)):
        cost += TSP_all[p] - TSP[p][i]
    M[i] = cost / len(P)

for i in range(len(I)):
    CL[i] = alpha * R[i] + (1 - alpha) * M[i]



for i in range(8):
    print(R[i], end=', ')
print('\n')
for i in range(8):
    print(M[i], end=', ')
print('\n')
for i in range(8):
    print(CL[i], end=', ')

0, 0, 0, 0, 1.1, 0.9333333333333333, 0.6, 0, 

399.5, 593.0, 742.5, 833.0, 910.5, 1110.0, 1151.5, 1356.0, 

199.75, 296.5, 371.25, 416.5, 455.8, 555.4666666666667, 576.05, 678.0, 

In [ ]:
start = 0
end = 10
width = end - start

R_converted = np.array(R)
res1 = (R_converted - R_converted.min())/(R_converted.max() - R_converted.min()) * width + start
R = res1.tolist()

M_converted = np.array(M)
res2 = (M_converted - M_converted.min())/(M_converted.max() - M_converted.min()) * width + start
M = res2.tolist()

CL_converted = np.array(CL)
TC = np.mean(CL_converted)

In [ ]:
###Solve the Linear Program

n_depots = 2
n_customers = 8
CC = 8

def make_var_b(p, i):
    return LpVariable(f"b_{p}_{i+n_depots}", lowBound=0, upBound=1, cat='Integer') # Satisfy constraint (26) already

c_pi = [[1092, 667, 816, 700, 158, 393, 650, 509], [187, 430, 669, 782, 856, 758, 357, 719]]

b_pi = [
    [make_var_b(i, j) for j in range(n_customers)]
    for i in range(n_depots)
]

flat_b_pi= [item for sublist in b_pi for item in sublist]
flat_c_pi= [item for sublist in c_pi for item in sublist]

#Define Linear Program
prob = LpProblem("Clustering", LpMinimize)

#Objective Function (22)
constraint22 = lpDot(flat_b_pi, flat_c_pi)
prob += constraint22
# print(constraint22)

#Constraint (23)
for p in range(n_depots):
  prob += lpSum(b_pi[p]) <= CC

#Constraint (24)
for i in range(n_customers):
  prob += b_pi[0][i] + b_pi[1][i] == 1

#Constraint(25)
CL_div_CC = []
for num in CL:
  CL_div_CC.append(num/CC)
for p in range(n_depots):
  prob += lpDot(b_pi[p], CL_div_CC) <= TC

prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
        print(v.name, "=", v.varValue)

Status: Optimal
b_0_2 = 0.0
b_0_3 = 0.0
b_0_4 = 0.0
b_0_5 = 1.0
b_0_6 = 1.0
b_0_7 = 1.0
b_0_8 = 0.0
b_0_9 = 1.0
b_1_2 = 1.0
b_1_3 = 1.0
b_1_4 = 1.0
b_1_5 = 0.0
b_1_6 = 0.0
b_1_7 = 0.0
b_1_8 = 1.0
b_1_9 = 0.0


In [ ]:
number_of_cluster = n_depots
cluster_1 = []
cluster_2 = []
cluster_list = [cluster_1, cluster_2]

for v in prob.variables():
  l = f'{v}'.split('_')
  p = int(l[1])
  i = int(l[2])
  if v.varValue == 1:
    cluster_list[p].append(i)

cluster = []
for c in cluster_list:
  cluster.append(np.array(c))

cluster_1 = cluster[0] + 2
cluster_2 = cluster[1] + 2

T = np.array([0, 1, 2])

cluster_1 = cluster_list[0]
cluster_2 = cluster_list[1]

In [ ]:
!pip install dijkstar
from dijkstar import Graph, find_path

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/parser.py", line 12, in <module>
    from pip._internal.configuration import Configuration, ConfigurationError
  File "/usr/local/lib/python3.7/dist-packages/pip/_intern

In [ ]:
##First for loop of part 2

d_list = np.array([
    [0, 1007, 1092, 667, 816, 700, 158, 393, 650, 509],
    [1007, 0, 187, 430, 669, 782, 856, 758, 357, 719],
    [1092, 187, 0, 447, 593, 734, 935, 788, 454, 872],
    [667, 430, 447, 0, 329, 382, 509, 342, 192, 611],
    [816, 669, 593, 329, 0, 169, 676, 423, 521, 914],
    [700, 782, 734, 382, 169, 0, 576, 319, 562, 888],
    [158, 856, 935, 509, 676, 576, 0, 258, 499, 449],
    [393, 758, 788, 342, 423, 319, 258, 0, 429, 609],
    [650, 357, 454, 192, 521, 562, 499, 429, 0, 456],
    [509, 719, 872, 611, 914, 888, 449, 609, 456, 0]
    ])
print(d_list.shape)

graph = Graph()

for i, d in enumerate(d_list):
  for j, cost in enumerate(d):
    if i != j:
      graph.add_edge(i, j, d_list[i, j])

R_l= []
for p, C in enumerate(cluster):
  for customer in C:
    path_info = find_path(graph, p, customer)
    print(path_info)
    path = getattr(path_info, 'nodes')
    R_l.append(path)

print(R_l)

R = set(tuple(i) for i in R_l)
print(R)

(10, 10)
PathInfo(nodes=[0, 5], edges=[700], costs=[700], total_cost=700)
PathInfo(nodes=[0, 6], edges=[158], costs=[158], total_cost=158)
PathInfo(nodes=[0, 7], edges=[393], costs=[393], total_cost=393)
PathInfo(nodes=[0, 9], edges=[509], costs=[509], total_cost=509)
PathInfo(nodes=[1, 2], edges=[187], costs=[187], total_cost=187)
PathInfo(nodes=[1, 3], edges=[430], costs=[430], total_cost=430)
PathInfo(nodes=[1, 4], edges=[669], costs=[669], total_cost=669)
PathInfo(nodes=[1, 8], edges=[357], costs=[357], total_cost=357)
[[0, 5], [0, 6], [0, 7], [0, 9], [1, 2], [1, 3], [1, 4], [1, 8]]
{(1, 2), (1, 3), (0, 7), (1, 4), (0, 6), (1, 8), (0, 5), (0, 9)}


In [ ]:
# g1 policy
g1_clus1 = np.empty((len(cluster_1), len(T)))
for i in range(len(cluster_1)):
    for j in range(len(T)):
        yih_g1 = 0
        d_ih = 0
        Iit_g1 = data[cluster_1[i]][3][0] + yih_g1 - d_ih
        if data[cluster_1[i]][4][j] >= Iit_g1:
            yit_g1 = data[cluster_1[i]][2] - Iit_g1
        else:
            yit_g1 = 0
        g1_clus1[i, j] = yit_g1
g1_clus1 = g1_clus1.astype(int)
print(g1_clus1) #[[yi0(g1), yi1(g1), yi2(g1)],[cust 2], [cust 3], ...]

g1_clus2 = np.empty((len(cluster_2), len(T)))
for i in range(len(cluster_2)):
    for j in range(len(T)):
        yih_g1 = 0
        d_ih = 0
        Iit_g1 = data[cluster_1[i]][3][0] + yih_g1 - d_ih
        if data[cluster_2[i]][4][j] >= Iit_g1:
            yit_g1 = data[cluster_2[i]][2] - Iit_g1
        else:
            yit_g1 = 0
        g1_clus2[i, j] = yit_g1
g1_clus2 = g1_clus2.astype(int)
print(g1_clus2)

[[  0   0   0]
 [ 49  49  49]
 [160 160 160]
 [  0   0   0]]
[[152 152 152]
 [ 59  59  59]
 [  0   0   0]
 [  0   0   0]]


In [ ]:
# g2 policy
g2_clus1 = np.empty((len(cluster_1), len(T)))
for i in range(len(cluster_1)):
    for j in range(len(T)):
        yih_g2 = 0
        d_ih = 0
        Iit_g2 = data[cluster_1[i]][3][0] + yih_g2 - d_ih
        if data[cluster_1[i]][4][j] >= Iit_g2:
            yit_g2 = data[cluster_1[i]][4][j] - Iit_g2
        else:
            yit_g2 = 0
        g2_clus1[i, j] = yit_g2
g2_clus1 = g2_clus1.astype(int)
print(g2_clus1) #[[yi0(g2), yi1(g2), yi2(g2)],[cust 2], [cust 3], ...]

g2_clus2 = np.empty((len(cluster_2), len(T)))
for i in range(len(cluster_2)):
    for j in range(len(T)):
        yih_g2 = 0
        d_ih = 0
        Iit_g2 = data[cluster_1[i]][3][0] + yih_g2 - d_ih
        if data[cluster_2[i]][4][j] >= Iit_g2:
            yit_g2 = data[cluster_2[i]][4][j] - Iit_g2
        else:
            yit_g2 = 0
        g2_clus2[i, j] = yit_g2
g2_clus2 = g2_clus2.astype(int)
print(g2_clus2)

[[  0   0   0]
 [ 33  11  22]
 [140 140   0]
 [  0   0   0]]
[[147  82  17]
 [ 24  24  24]
 [  0   0   0]
 [  0   0   0]]


In [ ]:
# g3 policy
g3_clus1 = np.empty((len(cluster_1), len(T)))
for i in range(len(cluster_1)):
    for j in range(len(T)):
        yih_g3 = 0
        d_ih = 0
        Iit_g3 = data[cluster_1[i]][3][0] + yih_g3 - d_ih
        if data[cluster_1[i]][4][j] >= Iit_g3:
            yit_g3 = max(data[cluster_1[i]][4][j] - Iit_g3, data[cluster_1[i]][3][0] - Iit_g3)
        else:
            yit_g3 = 0
        g3_clus1[i, j] = yit_g3
g3_clus1 = g3_clus1.astype(int)
print(g3_clus1) #[[yi0(g3), yi1(g3), yi2(g3)],[cust 2], [cust 3], ...]

g3_clus2 = np.empty((len(cluster_2), len(T)))
for i in range(len(cluster_2)):
    for j in range(len(T)):
        yih_g3 = 0
        d_ih = 0
        Iit_g3 = data[cluster_2[i]][3][0] + yih_g3 - d_ih
        if data[cluster_2[i]][4][j] >= Iit_g3:
            yit_g3 = max(data[cluster_2[i]][4][j] - Iit_g3, data[cluster_2[i]][3][0] - Iit_g3)
        else:
            yit_g3 = 0
        g3_clus2[i, j] = yit_g3
g3_clus2 = g3_clus2.astype(int)
print(g3_clus2)

[[  0   0   0]
 [ 33  11  22]
 [140 140   0]
 [  0   0   0]]
[[65  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [90  0  0]]


In [ ]:
# g4 policy
cl_avg = np.sum(CL) / len(CL)
g4_clus1 = np.empty((len(cluster_1), len(T)))
for i in range(len(cluster_1)):
    for j in range(len(T)):
        yih_g4 = 0
        d_ih = 0
        Iit_g4 = data[cluster_1[i]][3][0] + yih_g4 - d_ih
        if (data[cluster_1[i]][4][j] >= Iit_g4 and CL[(np.array(cluster_1) - 2)[i]] >= cl_avg):
            yit_g4 = min(2*data[cluster_1[i]][4][j], data[cluster_1[i]][2])
        elif (data[cluster_1[i]][4][j] >= Iit_g4 and CL[(np.array(cluster_1) - 2)[i]] < cl_avg):
            yit_g4 = data[cluster_1[i]][4][j]
        else:
            yit_g4 = 0
        g4_clus1[i, j] = yit_g4
g4_clus1 = g4_clus1.astype(int)
print(g4_clus1) #[[yi0(g4), yi1(g4), yi2(g4)],[cust 2], [cust 3], ...]

g4_clus2 = np.empty((len(cluster_2), len(T)))
for i in range(len(cluster_2)):
    for j in range(len(T)):
        yih_g4 = 0
        d_ih = 0
        Iit_g4 = data[cluster_1[i]][3][0] + yih_g4 - d_ih
        if (data[cluster_2[i]][4][j] >= Iit_g4 and CL[(np.array(cluster_2) - 2)[i]] >= cl_avg):
            yit_g4 = min(2*data[cluster_2[i]][4][j], data[cluster_2[i]][2])
        elif (data[cluster_2[i]][4][j] >= Iit_g4 and CL[(np.array(cluster_2) - 2)[i]] < cl_avg):
            yit_g4 = data[cluster_2[i]][4][j]
        else:
            yit_g4 = 0
        g4_clus2[i, j] = yit_g4
g4_clus2 = g4_clus2.astype(int)
print(g4_clus2)

[[  0   0   0]
 [ 60  44  60]
 [300 300 280]
 [  0   0   0]]
[[195 130  65]
 [ 35  35  35]
 [  0   0   0]
 [  0   0   0]]
